<a href="https://colab.research.google.com/github/sravan1320/RAG/blob/main/RAG_MultiModal_LLAMA_LLAVA_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Reference : https://github.com/sravan1320/langchain_examples/blob/master/cookbook/Semi_structured_multi_modal_RAG_LLaMA2.ipynb

In [ ]:
# ! pip install langchain unstructured[all-docs] pydantic lxml pdfminer.six pdf2image

In [1]:
# !pip install langchain unstructured==0.11.5 unstructured-inference==0.7.19 pydantic lxml pikepdf pypdf wrapt pdfminer.six unstructured_pytesseract

!pip install langchain unstructured==0.11.6 unstructured-inference==0.7.18 pydantic lxml pikepdf==8.8.0 pypdf==3.17.1 wrapt pdfminer.six==20221105 unstructured_pytesseract==0.3.12 pdf2image==1.16.3 lida==0.0.10 pillow==10.0.1 typing-extensions==4.5.0 accelerate bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup

## Install System Dependencies

In [2]:
!apt-get install poppler-utils
!apt-get install libmagic-dev
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.3 [186 kB]
Fetched 186 kB in 0s (424 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121658 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.3_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.3) ...
Setting up poppler-utils (22.02.0-2ubuntu0.3) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libmagic-dev
0 upgraded, 1 newly installed, 

## Validating Installation

In [3]:
# !pip freeze> req1.txt
# !pip list unstructured*

In [4]:
# from unstructured.partition.auto import partition

# elements = partition("Bunge_2022.pdf")
# print("\n\n".join([str(el) for el in elements]))

In [5]:
# from typing import Any
# from pydantic import BaseModel
# from unstructured.partition.pdf import partition_pdf
# elements = partition_pdf(filename="Bunge_2022.pdf", infer_table_structure=True, extract_images_in_pdf=True, url=None)

In [6]:
# elements

In [7]:
from typing import Any
from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf

# Path to save images
path = "docs/"
output_path= "docs/Output/"

# Get elements
raw_pdf_elements = partition_pdf(
    filename=path + "Bunge_2022.pdf",
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=output_path,
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# Unique_categories will have unique elements
# TableChunk if Table > max chars set above
unique_categories = set(category_counts.keys())
category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 258,
 "<class 'unstructured.documents.elements.Table'>": 114,
 "<class 'unstructured.documents.elements.TableChunk'>": 6}

In [9]:
class Element(BaseModel):
    type: str
    text: Any


# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

120
258


## TO Run Model in Local

model = ChatOllama(model="llama2:13b-chat")

## To Run Model from hugging face
### In our lets use model from hugging face

!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets

In [10]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    logging,
)

# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load the entire model on the GPU 0
device_map = {"": 0}

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [13]:
def get_prompt_response(text):
  # Define your prompt
  prompt_text = """You are an assistant tasked with summarizing tables and text. \
  Give a concise summary of the table or text. Table or text chunk: {element} """

  # Instantiate the prompt template
  prompt_template = ChatPromptTemplate.from_template(prompt_text)

  # Format the prompt with the provided element
  formatted_prompt = prompt_template.format(element=str(text))

  # Tokenize the formatted prompt
  inputs = tokenizer(formatted_prompt, return_tensors="pt", max_length=1024, truncation=True)

  # Generate the model response
  with torch.no_grad():
      output = model(**inputs)

  # Get the generated text from the output
  # generated_text = tokenizer.decode(output.logits[0], skip_special_tokens=True)
  generated_text = tokenizer.decode(output.logits[0].argmax(dim=-1).tolist(), skip_special_tokens=True)

  return generated_text

In [14]:
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [15]:
# # Apply to text
texts = [i.text for i in text_elements if i.text != ""]
# text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
# List to store generated responses
text_summaries = [get_prompt_response(text) for text in texts]

In [16]:
# Apply to tables
tables = [i.text for i in table_elements]
# table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})
table_summaries = [get_prompt_response(table) for table in tables]

# Clone Repo & LLAVA Huggingface Repo

In [30]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 14571, done.
remote: Counting objects: 100% (4359/4359), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 14571 (delta 4206), reused 4133 (delta 4079), pack-reused 10212
Receiving objects: 100% (14571/14571), 16.35 MiB | 20.32 MiB/s, done.
Resolving deltas: 100% (10182/10182), done.


In [18]:
# !git lfs install
# !git clone https://huggingface.co/mys/ggml_llava-v1.5-7b

from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="mys/ggml_llava-v1.5-7b", filename="ggml-model-q4_k.gguf")
#Model downloaded into
# '/root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/ggml-model-q4_k.gguf'
# '/root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/mmproj-model-f16.gguf'
hf_hub_download(repo_id="mys/ggml_llava-v1.5-7b", filename="mmproj-model-f16.gguf")

ggml-model-q4_k.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

mmproj-model-f16.gguf:   0%|          | 0.00/624M [00:00<?, ?B/s]

'/root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/mmproj-model-f16.gguf'

In [45]:
# !rm -rvf llama.cpp/build

In [57]:
!cd llama.cpp
!make -C llama.cpp

make: Entering directory '/content/llama.cpp'
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi
I NVCCFLAGS:  
I LDFLAGS:    
I CC:        cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:       g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

cc  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qu

In [60]:
!./llama.cpp/llava-cli


usage: ./llama.cpp/llava-cli [options]

options:
  -h, --help            show this help message and exit
      --version         show version and build info
  -i, --interactive     run in interactive mode
  --interactive-first   run in interactive mode and wait for input right away
  -ins, --instruct      run in instruction mode (use with Alpaca models)
  -cml, --chatml        run in chatml mode (use with ChatML-compatible models)
  --multiline-input     allows you to write or paste multiple lines without ending each in '\'
  -r PROMPT, --reverse-prompt PROMPT
                        halt generation at PROMPT, return control in interactive mode
                        (can be specified more than once for multiple prompts).
  --color               colorise output to distinguish prompt and user input from generations
  -s SEED, --seed SEED  RNG seed (default: -1, use random seed for < 0)
  -t N, --threads N     number of threads to use during generation (default: 2)
  -tb N, --threads-b

In [63]:
%%bash

# Define the directory containing the images
# IMG_DIR=~/Desktop/Papers/LLaVA/
IMG_DIR='docs/Output/'

# Loop through each image in the directory
for img in "${IMG_DIR}"*.jpg; do
    # Extract the base name of the image without extension
    base_name=$(basename "$img" .jpg)

    # Define the output file name based on the image name
    output_file="${IMG_DIR}${base_name}.txt"

    # '/root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/ggml-model-q4_k.gguf'
# '/root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/mmproj-model-f16.gguf'

    # Execute the command and save the output to the defined output file
    ! ./llama.cpp/llava-cli -m /root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/ggml-model-q4_k.gguf --mmproj /root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/mmproj-model-f16.gguf --temp 0.1 -p "Describe the image in detail. Be specific about graphs, such as bar plots." --image "$img" > "$output_file"

done

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--mys--ggml_llava-v1.5-7b/snapshots/9b713a64048c7f982ec3969e60e9f61f7a2730c2/ggml-model-q4_k.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32       

CalledProcessError: ignored

In [89]:
import glob
import os

path= "/content/docs/Output/"

# Get all .txt files in the directory
file_paths = glob.glob(os.path.expanduser(os.path.join(path, "*.txt")))

# Read each file and store its content in a list
img_summaries = []
for file_path in file_paths:
    with open(file_path, "r") as file:
        img_summaries.append(file.read())

# Clean up residual logging
cleaned_img_summary = [
    s.split("clip_model_load: model name:   openai/clip-vit-large-patch14-336\nclip_model_load: description:  image encoder for LLaVA\nclip_model_load: GGUF version: 2\nclip_model_load: alignment:    32\nclip_model_load: n_tensors:    377\nclip_model_load: n_kv:         18\nclip_model_load: ftype:        f16\n\nclip_model_load: text_encoder:   0\nclip_model_load: vision_encoder: 1\nclip_model_load: llava_projector:  1\nclip_model_load: model size:     595.63 MB\nclip_model_load: metadata size:  0.14 MB\nclip_model_load: total allocated memory: 195.98 MB\n\nencode_image_with_clip: image encoded in 17338.56 ms by CLIP (   30.10 ms per image patch)\n\n", 1)[1].strip()
    for s in img_summaries
]

In [90]:
cleaned_img_summary

["The image displays a graph showing the performance of two companies, one in the United States and the other in Canada. The graph is a line graph, with each company's performance represented by a different color. The line for the United States company is green, while the line for the Canadian company is blue.\n\nThe graph shows the performance of the two companies over time, with the dates displayed on the x-axis. The y-axis represents the performance of the companies, with the values increasing from left to right. The graph provides a clear visual representation of the performance of these two companies, allowing for easy comparison and analysis."]

In [66]:
!pip install gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 15.5 MB/s eta 0:00:00


In [69]:
!pip install chromadb

In [70]:
import uuid

from langchain.embeddings import GPT4AllEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma
from langchain_core.documents import Document

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries", embedding_function=GPT4AllEmbeddings()
)

# The storage layer for the parent documents
store = InMemoryStore()  # <- Can we extend this to images
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [91]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(text_summaries)
]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [
    Document(page_content=s, metadata={id_key: table_ids[i]})
    for i, s in enumerate(table_summaries)
]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

# Add images
img_ids = [str(uuid.uuid4()) for _ in cleaned_img_summary]
summary_img = [
    Document(page_content=s, metadata={id_key: img_ids[i]})
    for i, s in enumerate(cleaned_img_summary)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(
    list(zip(img_ids, cleaned_img_summary))
)  # Store the image summary as the raw document

In [92]:
from langchain_core.runnables import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Option 1: LLM
# model = ChatOllama(model="llama2:13b-chat")
# Option 2: Multi-modal LLM
# model = LLaVA

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [2]:
def get_response(text):
  # Define your prompt
  template = """Answer the question based only on the following context, which can include text and tables:
  {context}
  Question: {question}
  """

  # Instantiate the prompt template
  prompt_template = ChatPromptTemplate.from_template(template)

  # Format the prompt with the provided element
  formatted_prompt = prompt_template.format(context=retriever, question=text)

  # Tokenize the formatted prompt
  inputs = tokenizer(formatted_prompt, return_tensors="pt", max_length=1024, truncation=True)

  # Generate the model response
  with torch.no_grad():
      output = model(**inputs)

  # Get the generated text from the output
  # generated_text = tokenizer.decode(output.logits[0], skip_special_tokens=True)
  generated_text = tokenizer.decode(output.logits[0].argmax(dim=-1).tolist(), skip_special_tokens=True)

  return generated_text

In [4]:
from langchain.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
text="What is the performance of LLaVa across across multiple image domains / subjects?"
get_response(text)

ModuleNotFoundError: ignored

In [72]:
cleaned_img_summary

[]